## Import

In [9]:
# for read data
import os
import numpy as np
import pandas as pd

# model
from catboost import CatBoostClassifier

# Out of Fold
from sklearn.metrics import log_loss  
from sklearn.model_selection import StratifiedKFold

# for save
import joblib

## Read data

In [15]:
path = (os.path.abspath("./input"))

# 비정형 데이터인 text를 Word2Vec한 Feature만을 사용한다.
X_train = pd.read_csv(path +'/feature_train_W2V.csv', encoding='cp949')
X_test = pd.read_csv(path +'/feature_test_W2V.csv', encoding='cp949')
y_train = pd.read_csv(path +'/y_train.csv', encoding='cp949').group

In [16]:
train_ID, test_ID = X_train.custid, X_test.custid
del X_train['custid'], X_test['custid']

## Train & Out of Fold
Catboost는 Hyperparameter Tunning이 중요하지 않기에 학습횟수를 크게하여 학습시킨다.<br>
학습과 동시에 Submission data를 생성한다.

In [17]:
SKF = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

In [18]:
model = CatBoostClassifier(iterations=1000, learning_rate = 0.03, bootstrap_type ='Bayesian',
                           objective='MultiClass', devices='0:1', task_type="GPU", random_state=0)

In [19]:
cat_pred = np.zeros((X_test.shape[0], 8))
loss_list = []
for tr_idx, val_idx in SKF.split(X_train, y_train):
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    model.fit(tr_x, tr_y, early_stopping_rounds = 1000)
    pred = model.predict_proba(val_x)
    loss = log_loss(val_y, pred)
    loss_list.append(loss)
    
    sub_pred = np.array(model.predict_proba(X_test)) / 5  # averaging
    cat_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 Log Loss는 {np.mean(loss_list)}')

0:	learn: 2.0515239	total: 63.8ms	remaining: 1m 3s
1:	learn: 2.0258377	total: 124ms	remaining: 1m 1s
2:	learn: 2.0026637	total: 185ms	remaining: 1m 1s
3:	learn: 1.9816590	total: 246ms	remaining: 1m 1s
4:	learn: 1.9621520	total: 307ms	remaining: 1m 1s
5:	learn: 1.9440413	total: 367ms	remaining: 1m
6:	learn: 1.9268681	total: 429ms	remaining: 1m
7:	learn: 1.9113160	total: 489ms	remaining: 1m
8:	learn: 1.8969742	total: 550ms	remaining: 1m
9:	learn: 1.8835558	total: 609ms	remaining: 1m
10:	learn: 1.8707710	total: 668ms	remaining: 1m
11:	learn: 1.8585660	total: 726ms	remaining: 59.8s
12:	learn: 1.8470306	total: 786ms	remaining: 59.7s
13:	learn: 1.8360315	total: 846ms	remaining: 59.6s
14:	learn: 1.8258528	total: 904ms	remaining: 59.4s
15:	learn: 1.8158923	total: 965ms	remaining: 59.3s
16:	learn: 1.8064766	total: 1.02s	remaining: 59.3s
17:	learn: 1.7977756	total: 1.08s	remaining: 59.2s
18:	learn: 1.7892506	total: 1.14s	remaining: 59.1s
19:	learn: 1.7813156	total: 1.2s	remaining: 59s
20:	learn:

164:	learn: 1.4739300	total: 9.83s	remaining: 49.7s
165:	learn: 1.4732253	total: 9.89s	remaining: 49.7s
166:	learn: 1.4723743	total: 9.94s	remaining: 49.6s
167:	learn: 1.4715923	total: 10s	remaining: 49.5s
168:	learn: 1.4707155	total: 10.1s	remaining: 49.5s
169:	learn: 1.4698392	total: 10.1s	remaining: 49.4s
170:	learn: 1.4690433	total: 10.2s	remaining: 49.4s
171:	learn: 1.4681840	total: 10.2s	remaining: 49.3s
172:	learn: 1.4673445	total: 10.3s	remaining: 49.3s
173:	learn: 1.4664084	total: 10.4s	remaining: 49.2s
174:	learn: 1.4654843	total: 10.4s	remaining: 49.2s
175:	learn: 1.4649158	total: 10.5s	remaining: 49.1s
176:	learn: 1.4639505	total: 10.5s	remaining: 49.1s
177:	learn: 1.4631529	total: 10.6s	remaining: 49s
178:	learn: 1.4621355	total: 10.7s	remaining: 48.9s
179:	learn: 1.4611914	total: 10.7s	remaining: 48.9s
180:	learn: 1.4603462	total: 10.8s	remaining: 48.8s
181:	learn: 1.4595501	total: 10.9s	remaining: 48.8s
182:	learn: 1.4586379	total: 10.9s	remaining: 48.7s
183:	learn: 1.45

325:	learn: 1.3738338	total: 19.1s	remaining: 39.5s
326:	learn: 1.3731830	total: 19.2s	remaining: 39.4s
327:	learn: 1.3725643	total: 19.2s	remaining: 39.4s
328:	learn: 1.3719382	total: 19.3s	remaining: 39.3s
329:	learn: 1.3714656	total: 19.3s	remaining: 39.3s
330:	learn: 1.3708709	total: 19.4s	remaining: 39.2s
331:	learn: 1.3704182	total: 19.5s	remaining: 39.1s
332:	learn: 1.3699963	total: 19.5s	remaining: 39.1s
333:	learn: 1.3693876	total: 19.6s	remaining: 39s
334:	learn: 1.3688580	total: 19.6s	remaining: 39s
335:	learn: 1.3682287	total: 19.7s	remaining: 38.9s
336:	learn: 1.3679372	total: 19.7s	remaining: 38.8s
337:	learn: 1.3672461	total: 19.8s	remaining: 38.8s
338:	learn: 1.3668342	total: 19.8s	remaining: 38.7s
339:	learn: 1.3663888	total: 19.9s	remaining: 38.6s
340:	learn: 1.3658599	total: 20s	remaining: 38.6s
341:	learn: 1.3655095	total: 20s	remaining: 38.5s
342:	learn: 1.3649459	total: 20.1s	remaining: 38.4s
343:	learn: 1.3645106	total: 20.1s	remaining: 38.4s
344:	learn: 1.364071

487:	learn: 1.2953916	total: 28.3s	remaining: 29.6s
488:	learn: 1.2948888	total: 28.3s	remaining: 29.6s
489:	learn: 1.2942872	total: 28.4s	remaining: 29.5s
490:	learn: 1.2937557	total: 28.4s	remaining: 29.5s
491:	learn: 1.2932642	total: 28.5s	remaining: 29.4s
492:	learn: 1.2928405	total: 28.5s	remaining: 29.4s
493:	learn: 1.2924928	total: 28.6s	remaining: 29.3s
494:	learn: 1.2920330	total: 28.7s	remaining: 29.2s
495:	learn: 1.2916315	total: 28.7s	remaining: 29.2s
496:	learn: 1.2912655	total: 28.8s	remaining: 29.1s
497:	learn: 1.2909710	total: 28.8s	remaining: 29.1s
498:	learn: 1.2905525	total: 28.9s	remaining: 29s
499:	learn: 1.2900897	total: 28.9s	remaining: 28.9s
500:	learn: 1.2897764	total: 29s	remaining: 28.9s
501:	learn: 1.2894166	total: 29.1s	remaining: 28.8s
502:	learn: 1.2891169	total: 29.1s	remaining: 28.8s
503:	learn: 1.2886377	total: 29.2s	remaining: 28.7s
504:	learn: 1.2881585	total: 29.2s	remaining: 28.6s
505:	learn: 1.2877982	total: 29.3s	remaining: 28.6s
506:	learn: 1.28

648:	learn: 1.2296905	total: 37.3s	remaining: 20.2s
649:	learn: 1.2293856	total: 37.4s	remaining: 20.1s
650:	learn: 1.2290737	total: 37.4s	remaining: 20.1s
651:	learn: 1.2286031	total: 37.5s	remaining: 20s
652:	learn: 1.2283411	total: 37.5s	remaining: 19.9s
653:	learn: 1.2279877	total: 37.6s	remaining: 19.9s
654:	learn: 1.2275508	total: 37.6s	remaining: 19.8s
655:	learn: 1.2270526	total: 37.7s	remaining: 19.8s
656:	learn: 1.2266754	total: 37.8s	remaining: 19.7s
657:	learn: 1.2263448	total: 37.8s	remaining: 19.7s
658:	learn: 1.2259309	total: 37.9s	remaining: 19.6s
659:	learn: 1.2254245	total: 37.9s	remaining: 19.5s
660:	learn: 1.2249559	total: 38s	remaining: 19.5s
661:	learn: 1.2245399	total: 38s	remaining: 19.4s
662:	learn: 1.2242521	total: 38.1s	remaining: 19.4s
663:	learn: 1.2239637	total: 38.1s	remaining: 19.3s
664:	learn: 1.2235488	total: 38.2s	remaining: 19.2s
665:	learn: 1.2232179	total: 38.3s	remaining: 19.2s
666:	learn: 1.2228284	total: 38.3s	remaining: 19.1s
667:	learn: 1.2223

...

346:	learn: 1.3638863	total: 20.6s	remaining: 38.8s
347:	learn: 1.3634079	total: 20.7s	remaining: 38.7s
348:	learn: 1.3629006	total: 20.7s	remaining: 38.6s
349:	learn: 1.3624069	total: 20.8s	remaining: 38.6s
350:	learn: 1.3617891	total: 20.8s	remaining: 38.5s
351:	learn: 1.3612387	total: 20.9s	remaining: 38.5s
352:	learn: 1.3607091	total: 20.9s	remaining: 38.4s
353:	learn: 1.3602690	total: 21s	remaining: 38.3s
354:	learn: 1.3598706	total: 21.1s	remaining: 38.3s
355:	learn: 1.3593036	total: 21.1s	remaining: 38.2s
356:	learn: 1.3589484	total: 21.2s	remaining: 38.1s
357:	learn: 1.3584257	total: 21.2s	remaining: 38.1s
358:	learn: 1.3577331	total: 21.3s	remaining: 38s
359:	learn: 1.3570239	total: 21.3s	remaining: 38s
360:	learn: 1.3565730	total: 21.4s	remaining: 37.9s
361:	learn: 1.3561667	total: 21.5s	remaining: 37.8s
362:	learn: 1.3556305	total: 21.5s	remaining: 37.8s
363:	learn: 1.3552696	total: 21.6s	remaining: 37.7s
364:	learn: 1.3548644	total: 21.6s	remaining: 37.6s
365:	learn: 1.3544

506:	learn: 1.2888239	total: 29.7s	remaining: 28.9s
507:	learn: 1.2883919	total: 29.8s	remaining: 28.9s
508:	learn: 1.2879476	total: 29.9s	remaining: 28.8s
509:	learn: 1.2873561	total: 29.9s	remaining: 28.8s
510:	learn: 1.2870139	total: 30s	remaining: 28.7s
511:	learn: 1.2866490	total: 30s	remaining: 28.6s
512:	learn: 1.2862684	total: 30.1s	remaining: 28.6s
513:	learn: 1.2857189	total: 30.2s	remaining: 28.5s
514:	learn: 1.2852751	total: 30.2s	remaining: 28.5s
515:	learn: 1.2848730	total: 30.3s	remaining: 28.4s
516:	learn: 1.2843344	total: 30.3s	remaining: 28.3s
517:	learn: 1.2838773	total: 30.4s	remaining: 28.3s
518:	learn: 1.2834325	total: 30.5s	remaining: 28.2s
519:	learn: 1.2830132	total: 30.5s	remaining: 28.2s
520:	learn: 1.2825716	total: 30.6s	remaining: 28.1s
521:	learn: 1.2821443	total: 30.6s	remaining: 28s
522:	learn: 1.2816885	total: 30.7s	remaining: 28s
523:	learn: 1.2813501	total: 30.7s	remaining: 27.9s
524:	learn: 1.2810435	total: 30.8s	remaining: 27.9s
525:	learn: 1.280537

666:	learn: 1.2242546	total: 38.7s	remaining: 19.3s
667:	learn: 1.2238709	total: 38.8s	remaining: 19.3s
668:	learn: 1.2233048	total: 38.9s	remaining: 19.2s
669:	learn: 1.2230325	total: 38.9s	remaining: 19.2s
670:	learn: 1.2226287	total: 39s	remaining: 19.1s
671:	learn: 1.2222591	total: 39s	remaining: 19s
672:	learn: 1.2217570	total: 39.1s	remaining: 19s
673:	learn: 1.2214586	total: 39.1s	remaining: 18.9s
674:	learn: 1.2210888	total: 39.2s	remaining: 18.9s
675:	learn: 1.2206881	total: 39.2s	remaining: 18.8s
676:	learn: 1.2202751	total: 39.3s	remaining: 18.7s
677:	learn: 1.2199695	total: 39.3s	remaining: 18.7s
678:	learn: 1.2195418	total: 39.4s	remaining: 18.6s
679:	learn: 1.2191593	total: 39.5s	remaining: 18.6s
680:	learn: 1.2188120	total: 39.5s	remaining: 18.5s
681:	learn: 1.2182721	total: 39.6s	remaining: 18.5s
682:	learn: 1.2179902	total: 39.6s	remaining: 18.4s
683:	learn: 1.2175475	total: 39.7s	remaining: 18.3s
684:	learn: 1.2170507	total: 39.7s	remaining: 18.3s
685:	learn: 1.216694

826:	learn: 1.1648557	total: 47.8s	remaining: 10s
827:	learn: 1.1645797	total: 47.9s	remaining: 9.94s
828:	learn: 1.1642376	total: 47.9s	remaining: 9.88s
829:	learn: 1.1638307	total: 48s	remaining: 9.82s
830:	learn: 1.1635252	total: 48s	remaining: 9.77s
831:	learn: 1.1631886	total: 48.1s	remaining: 9.71s
832:	learn: 1.1628015	total: 48.1s	remaining: 9.65s
833:	learn: 1.1622062	total: 48.2s	remaining: 9.59s
834:	learn: 1.1619457	total: 48.2s	remaining: 9.53s
835:	learn: 1.1615406	total: 48.3s	remaining: 9.47s
836:	learn: 1.1612697	total: 48.4s	remaining: 9.42s
837:	learn: 1.1609581	total: 48.4s	remaining: 9.36s
838:	learn: 1.1606651	total: 48.5s	remaining: 9.3s
839:	learn: 1.1603154	total: 48.5s	remaining: 9.24s
840:	learn: 1.1599774	total: 48.6s	remaining: 9.18s
841:	learn: 1.1595399	total: 48.6s	remaining: 9.12s
842:	learn: 1.1592936	total: 48.7s	remaining: 9.07s
843:	learn: 1.1589852	total: 48.7s	remaining: 9.01s
844:	learn: 1.1585511	total: 48.8s	remaining: 8.95s
845:	learn: 1.15830

988:	learn: 1.1096766	total: 56.9s	remaining: 632ms
989:	learn: 1.1093621	total: 56.9s	remaining: 575ms
990:	learn: 1.1090164	total: 57s	remaining: 517ms
991:	learn: 1.1086808	total: 57s	remaining: 460ms
992:	learn: 1.1084667	total: 57.1s	remaining: 402ms
993:	learn: 1.1081217	total: 57.1s	remaining: 345ms
994:	learn: 1.1077348	total: 57.2s	remaining: 287ms
995:	learn: 1.1074643	total: 57.3s	remaining: 230ms
996:	learn: 1.1070498	total: 57.3s	remaining: 172ms
997:	learn: 1.1066475	total: 57.4s	remaining: 115ms
998:	learn: 1.1064161	total: 57.4s	remaining: 57.5ms
999:	learn: 1.1061115	total: 57.5s	remaining: 0us
CatBoostClassifier의 5fold 평균 Log Loss는 1.5382609292516753


In [ ]:
# CAT_iterations = 1000, 1.55
# LGBM_iterations = 1000, 1.5382609292516753

## Make Submission File

In [7]:
pred = pd.DataFrame(cat_pred)
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
submissions = pd.concat([pd.Series(test_ID, name="ID"), pred] ,axis=1)

In [ ]:
sub_path = (os.path.abspath("./submission"))

fname = '/MCAT_FW2V.csv' # Model_FeatureSet
submissions.to_csv(sub_path+fname, index=False)
print("'{}' is ready to submit." .format(fname))